Na vypracování testu máte 120 minut. Očekávanými a hodnocenými výstupy jsou:
1.	Textový soubor s názvem váš_login.txt (např. pascepet.txt) obsahující zdrojový kód příkazů, kterými jste provedli zadání. Pokud máte zároveň něco zjistit nebo vypsat, zaznamenejte to přímo do souboru, ideálně jako komentář. **Tento soubor na konci své práce zašlete mailem na adresu <jan.hucin@profinit.eu> a podepište se svým celým jménem.**
2.	Existence a vlastnosti souborů, adresářů, tabulek, které jste zkopírovali či vytvořili při plnění zadání. Ty zhodnotíme přímo na clusteru.
V jednotlivých oblastech testu se hodnotí každý úkol nebo jeho část. Pokud si s nějakou částí zadání nebudete vědět rady, můžete ji přeskočit nebo zadání splnit bez této části, počet bodů se pak přiměřeně sníží.

### HDFS operace (3 body)
- Nastavte na svém uživatelském adresáři (/user/*váš_login*) v HDFS pro veřejnost právo read a execute. (Práva pro vlastníka a skupinu ponechte tak, jak jsou nyní.)

In [ ]:
hdfs dfs -chmod 755 /user/my_login
# nebo
hdfs dfs -chmod 755 .

- Ve svém uživatelském adresáři na HDFS (/user/*váš_login*) založte podadresář **chess_ratings** a nastavte u něj také pro veřejnost právo read a execute.

In [ ]:
hdfs dfs -mkdir chess_ratings
hdfs dfs -chmod 755 chess_ratings

- Zkopírujte z lokálního filesystemu na metacentru z podadresáře /home/pascepet/fel_bigdata/data/chess_ratings všechny soubory se jménem ve tvaru **standard_**___RRMM.csv___ na HDFS do podadresáře, který jste založili v předchozím kroku.

In [ ]:
hdfs dfs -put /home/pascepet/fel_bigdata/data/chess_ratings/standard_????.csv chess_ratings

> V dalších třech částech budete pracovat se stejnými daty. Na HDFS je adresář **/user/pascepet/data/chess_ratings_red** obsahující měsíční výpisy ratingové listiny hráčů šachu. Soubory jsou standardní textové s oddělovači (znak |), neobsahují hlavičky (názvy) sloupců. Každý řádek souboru obsahuje postupně tyto údaje: id a jméno hráče, stát, pohlaví, rating, počet odehraných her za minulé období, rok narození, rok a měsíc výpisu ratingové listiny.

### Práce s Hive (9 bodů)
Zkontrolujte, že máte založenou databázi Hive (název = váš login), a pokud ne, založte ji. Dále pracujte se svou databází.

In [ ]:
create database my_login;
use my_login;

- Vytvořte externí tabulku Hive *ratings_ext* založenou na datech z výše uvedeného adresáře (v rámečku). Názvy polí a typy zvolte vhodně podle svého uvážení.

In [ ]:
create external table ratings_ext (
id bigint,
name string,
fed string,
sex varchar(2),
rating smallint,
gms smallint,
birth_year smallint,
yr smallint,
mon tinyint
)
row format delimited
fields terminated by '|'
lines terminated by '\n'
stored as textfile
location '/user/pascepet/data/chess_ratings_red/';

- Vytvořte managed (interní) tabulku Hive *ratings* s formátem ORC a kompresí ZLIB. Do této tabulky přeneste data z externí tabulky, ale jen ty řádky, kde je jméno hráče neprázdné, rok narození větší než 0 a hodnota ratingu není null.

In [ ]:
create table ratings (
id bigint,
name string,
fed string,
sex varchar(2),
rating smallint,
gms smallint,
birth_year smallint,
yr smallint,
mon tinyint
)
stored as orc
tblproperties ("orc.compress"="ZLIB");

insert overwrite table ratings
select * from ratings_ext
where name<>'' and birth_year>0 and rating is not null;

Z výsledné tabulky *ratings* zjistěte pomocí SQL dotazu:
- Který stát má nejnižší věkový průměr ze všech záznamů ve výpisech ratingové listiny a kolik tento průměr je (věk počítejte k roku 2019)?

In [ ]:
select fed, 2019-avg(birth_year) vek_prum
from ratings
group by fed
order by vek_prum
limit 1;

- Kterých pět hráčů má největší (kladný) rozdíl mezi maximální a minimální hodnotou svého ratingu?

In [ ]:
select id, name, max(rating)-min(rating) rozdil
from ratings
group by id, name
order by rozdil desc
limit 5;

### Spark RDD (9 bodů)
- Načtěte obsah výše uvedeného adresáře (v rámečku) do RDD.

In [ ]:
rat = sc.textFile('/user/pascepet/data/chess_ratings/')

- Zjistěte, kolik unikátních hráčů je celkem ve výpisech uvedeno. Předpokládáme, že hráč je identifikován kombinací svého id a jména, záznamy s prázdným jménem neberte v úvahu.

In [ ]:
# redukuje data jen na id a jmeno, vyhodi duplicity
rat = rat.map(lambda line: (line.split('|')[0], line.split('|')[1]))
rat = rat.distinct()

# vyhodi se prazdna jmena
rat = rat.filter(lambda pair: pair[1]!='')

# nakesovat pro rychlejsi dalsi zpracovani
rat.cache()

# kolik unikatnich hracu ma aspon jeden zaznam
rat.count()
# --> 325 679 hracu

- Jméno každého hráče má několik částí, které jsou odděleny čárkou (např. „Navara, David“). Zjistěte, jaké jsou četnosti pro jednotlivé počty částí jména (kolik unikátních hráčů má jméno o jedné části, kolik jméno o dvou částech atd.).

In [ ]:
rat_name_part_num = rat.map(lambda pair: len(pair[1].split(',')))
rat_name_part_num = rat_name_part_num.map(lambda pocet: (pocet,1))
rat_freq = rat_name_part_num.reduceByKey(lambda a,b: a+b)
rat_freq.collect()
# --> [(1, 28970), (2, 295414), (3, 1295)]

- Zjistěte, v kolika případech je některá část jména rovna řetězci „Mustafa“. Pokud je v jednom jménu víc takových částí, počítá se každá část.

In [ ]:
rat_mustafa = rat.flatMap(lambda pair: pair[1].split(','))
rat_mustafa = rat_mustafa.map(lambda name: name.strip())
rat_mustafa.filter(lambda name: name=='Mustafa').count()
# --> 127

### Spark SQL (9 bodů)
- Načtěte obsah výše uvedeného adresáře (v rámečku) do DataFrame.

In [ ]:
from pyspark.sql.types import *

# definice schematu
schema_rat = StructType([
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("fed", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("rating", IntegerType(), True),
    StructField("gms", IntegerType(), True),
    StructField("birth_year", IntegerType(), True),
    StructField("yr", IntegerType(), True),
    StructField("mon", IntegerType(), True)])

# nacteni dat
ratDF = sqlContext.read \
    .format("com.databricks.spark.csv") \
    .options(header="true", delimiter = "|") \
    .load("/user/pascepet/data/chess_ratings_red", schema=schema_rat)

# je mozne nyni ocistit data
ratDF = ratDF.filter("name<>'' and birth_year>0")
# nebo
ratDF = ratDF.filter((ratDF.name!='') & (ratDF.birth_year>0))
ratDF.cache()

- Které všechny ženy z ČR (stát „CZE“) měly za aspoň jedno období ve výpisu rating vyšší než 2300?

In [ ]:
ratDF.filter(ratDF.sex=='F').filter(ratDF.fed=='CZE').filter(ratDF.rating>2300).select('id', 'name').distinct().show()
#| 304050|   Krupkova, Petra|
#| 308498|     Jackova, Jana|
#| 320242|Olsarova, Karolina|
#|1118080|     Worek, Joanna|

- Zjistěte, kteří tři hráči měli největší celkový počet odehraných her. Neberte v úvahu hráče s prázdným jménem.

In [ ]:
ratDFsum = ratDF.groupBy(ratDF.id, ratDF.name).sum('gms').toDF('id', 'name', 'gms_sum')
ratDFsum.orderBy(ratDFsum.gms_sum.desc()).show()
#| 5001277|Ramnathan, Balasu...|    300|
#| 5041198|   Subramanian, T.V.|    288|
#| 4101715|       Naumkin, Igor|    250|

- Vezmeme-li v úvahu jen hráče uvedené ve výpisech v prosinci 2017 i v prosinci 2018, jaké jsou průměrné ratingy těchto hráčů za každý z uvedených dvou měsíců?

In [ ]:
ratDF17 = ratDF.filter((ratDF.yr==2017) & (ratDF.mon==12)).select('id', 'rating').toDF('id', 'rat17')
ratDF18 = ratDF.filter((ratDF.yr==2018) & (ratDF.mon==12)).select('id', 'rating').toDF('id', 'rat18')
ratDF17.join(ratDF18, ratDF17.id==ratDF18.id).groupBy().avg('rat17', 'rat18').show()
#|        avg(rat17)|        avg(rat18)|
#+------------------+------------------+
#|1722.1577910035187|1724.5564836296196|